In [14]:
from genaisys.querying_functions import get_query_results, display_results
import sys
import os
import json

from IPython.display import display, HTML, clear_output
from ipywidgets import Dropdown, Text, Checkbox, Layout, Button, HBox, Output
from genaisys import init_openai_api, make_openai_api_call


# ============ SHARED STATE ============
user_histories = {"User01": [], "User02": [], "User03": []}
active_user = "User01"
conversation_active = True

# Create output widgets
conversation_output = Output()
debug_output = Output()
user_memory = True

# ============ FUNCTIONS ============

def chat_with_gpt(messages, user_message):
    try:
      namespace=""
      if "Pinecone" in user_message or "RAG" in user_message:
         # Determine the keyword
        if "Pinecone" in user_message:
            namespace="genaisys"
        elif "RAG" in user_message:
            namespace="data01"
        print(namespace)
        #define query text
        query_text=user_message
        # Retrieve query results
        query_results = get_query_results(query_text, namespace)
        # Process and display the results
        print("Processed query results:")
        qtext, target_id = display_results(query_results)
        print(qtext)
        #run task
        sc_input=qtext + " " + user_message
        mrole = "system"
        mcontent = "You are an assistant who executes the tasks you are asked to do."
        user_role = "user"
        task_response = make_openai_api_call(sc_input,mrole,mcontent,user_role)
        print(task_response)
        aug_output=namespace + ":" +task_response
      else:
        if user_memory:
                # Extract ALL user messages from the conversation history
                user_messages_content = [
                    msg["content"] for msg in messages
                    if msg["role"] == "user" and "content" in msg
                ]
                # Combine all extracted user messages into a single string
                combined_user_messages = " ".join(user_messages_content)
                # Add the current user_message to the combined text
                umessage = f"{combined_user_messages} {user_message}"
        else:
                umessage = user_message
        mrole = "system"
        mcontent = "You are an assistant who executes the tasks you are asked to do."
        user_role = "user"
        task_response = make_openai_api_call(umessage,mrole,mcontent,user_role)
        aug_output=task_response
      # Return the augmented output
      return aug_output
    except Exception as e:
        return f"An error occurred: {e}"

def save_conversation_history():
    """Save conversation history to JSON file."""
    filename = "conversation_history.json"
    with open(filename, 'w') as file:
        json.dump(user_histories, file, indent=4)
    display(HTML(f"<div style='color: green;'><strong>Saved to {filename}</strong></div>"))


def update_display():
    """Update the conversation display."""
    global active_user
    with debug_output:
        print(f"DEBUG: update_display() - Active user: {active_user}")
        print(f"DEBUG: History entries: {len(user_histories[active_user])}")

    with conversation_output:
        clear_output(wait=True)
        for entry in user_histories[active_user]:
            if entry['role'] == 'user':
                display(HTML(
                    f"<div style='margin-left: 20px; color: blue; margin-bottom: 5px;'>"
                    f"<strong>{active_user}:</strong> {entry['content']}</div>"
                ))
            elif entry['role'] == 'assistant':
                display(HTML(
                    f"<div style='margin-left: 20px; color: green; margin-bottom: 5px;'>"
                    f"<strong>Agent:</strong> {entry['content']}</div>"
                ))
        if conversation_active:
            display(VBox([user_selector, input_box, agent_checkbox]))  # Keep input box, selector, and checkbox visible if active


def chat(user_message):
    """Handle user input and generate response."""
    global active_user, conversation_active

    with debug_output:
        print(f"DEBUG: chat() called with: '{user_message}'")

    # Check for exit
    if user_message.lower() in ['exit', 'quit']:
        conversation_active = False
        with conversation_output:
            clear_output()
            display(HTML("<div style='color: red;'><strong>Conversation ended.</strong></div>"))
            save_conversation_history()
        return

    # Add user message to history
    user_histories[active_user].append({"role": "user", "content": user_message})

    # Generate response if agent is enabled
    if agent_checkbox.value:
        with debug_output:
            print("DEBUG: Generating agent response...")
        try:
            # The user history of this conversation and the user message are sent to the chat_with_gpt()
            response = chat_with_gpt(user_histories[active_user], user_message)
            with debug_output:
                print(f"DEBUG: Response: '{response[:100]}...'")
            # Append to the history of the response
            user_histories[active_user].append({"role": "assistant", "content": response})
        except Exception as e:
            with debug_output:
                print(f"DEBUG: Error: {e}")
            user_histories[active_user].append({"role": "assistant", "content": f"Error: {e}"})
    else:
        with debug_output:
            print("DEBUG: Agent disabled, skipping response")
    # Refresh the UI
    update_display()


def on_user_change(change):
    """Handle user selection change."""
    global active_user
    active_user = change['new']
    with debug_output:
        print(f"DEBUG: User changed to: {active_user}")
    update_display()


def handle_input(sender):
    """Handle input submission (Enter key or button)."""
    user_message = input_box.value
    with debug_output:
        print(f"DEBUG: Input received: '{user_message}'")

    if user_message and user_message.strip():
        input_box.value = ""  # Clear input
        chat(user_message)

# Function to save conversation history to a file
def save_conversation_history():
    filename = "conversation_history.json"  # Define the filename
    with open(filename, 'w') as file:
        json.dump(user_histories, file, indent=4)  # Write the user histories dictionary to the file in JSON format
    display(HTML(f"<div style='color: green;'><strong>Conversation history saved to {filename}.</strong></div>"))

# ============ UI WIDGETS ============

user_selector = Dropdown(
    options=["User01", "User02", "User03"],
    value=active_user,
    description='User:',
    layout=Layout(width='200px')
)
user_selector.observe(on_user_change, names='value')

input_box = Text(
    placeholder="Type your message here and press Enter or click Send",
    layout=Layout(width='400px'),
    continuous_update=False
)
input_box.observe(lambda change: handle_input(change) if change['name'] == 'value' and change['new'] else None, names='value')

send_button = Button(
    description='Send',
    button_style='primary',
    layout=Layout(width='80px')
)
send_button.on_click(lambda b: handle_input(b))

agent_checkbox = Checkbox(
    value=True,
    description='Agent',
    layout=Layout(width='100px')
)

# ============ DISPLAY INTERFACE ============

display(HTML("<h3>GenAI Chat Interface</h3>"))
display(user_selector)
display(HBox([input_box, send_button]))
display(agent_checkbox)
display(HTML("<hr><h4>Conversation:</h4>"))
display(conversation_output)
display(HTML("<hr><h4>Debug Output:</h4>"))
display(debug_output)

with debug_output:
    print("DEBUG: Interface ready.")

Dropdown(description='User:', layout=Layout(width='200px'), options=('User01', 'User02', 'User03'), value='Use…

Checkbox(value=True, description='Agent', layout=Layout(width='100px'))

Output()

Output()

In [8]:
display_conversation_history=True
summary=True
import os

if display_conversation_history == True or summary == True:
    file_path = "conversation_history.json"
    if os.path.exists(file_path):
        display_conversation_history = True
        summary = True
        print(f"The file {file_path} exists.")
    else:
        display_conversation_history = False
        summary = False
        print(f"The file {file_path} does not exist.")
        print("The conversation history will not be processed")


The file conversation_history.json exists.


In [15]:
from IPython.core.display import Markdown
# Load and summarize the conversation history
# Display option
display_conversation_history = True
if display_conversation_history:
  # File path
  file_path = 'conversation_history.json'
  # Open the file and read its content into the 'dialog' variable
  with open(file_path, 'r', encoding='utf-8') as file:
      dialog = json.load(file)  # Parse JSON content
  # Function to format JSON content as markdown
def summarize_conversation(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        dialog = file.read()
    conversation_history_json = json.loads(dialog)
    # Step 2: Construct dialog string from the JSON conversation history
    def construct_dialog(conversation_history_json):
        dialog = ""
        for user, messages in conversation_history_json.items():
            dialog += f"\n{user}:\n"
            for message in messages:
                role = message["role"]
                content = message["content"]
                dialog += f"- {role}: {content}\n"
        return dialog
    formatted_dialog = construct_dialog(conversation_history_json)
    # Step 3: Prepare the task for the summary
    mrole = "system"
    mcontent = "Your task is to read this JSON formatted text and summarize it."
    user_role = "user"
    task = f"Read this JSON formatted text and make a very detailed summary of it with a list of actions:\n{formatted_dialog}"
    # Step 4: Call the `make_openai_api_call` function
    task_response = make_openai_api_call(task, mrole, mcontent, user_role)
    # Step 5: Display the task response as Markdown
    display(Markdown(task_response))

if summary:
    # File path to the JSON file
    file_path = 'conversation_history.json'
    # Check if the file exists before calling the function
    if os.path.exists(file_path):
        summarize_conversation(file_path)
    else:
        print(f"File '{file_path}' does not exist. Please provide a valid file path.")


The JSON text involves a conversation between a user and an AI assistant regarding customer feedback and the application of AI memory systems to improve a travel agency's services. Here's a detailed summary with a list of actions:

### Summary:

1. **Customer Feedback Analysis:**
   - A customer expressed dissatisfaction with the travel agency, comparing it unfavorably to competitors and suggesting service improvements. The sentiment analysis score was low (0.2), indicating negative sentiment.

2. **Understanding AI Memory Systems:**
   - The user inquired about the CTO's mention of leveraging different kinds of memories, which was explained in the context of Retrieval-Augmented Generation (RAG). The AI assistant detailed various memory types:
     - **Short-term Memory:** Immediate context for processing queries.
     - **Long-term Memory:** Persistent storage for future access.
     - **Episodic Memory:** Storage of specific events or experiences.
     - **Semantic Memory:** General knowledge and facts.
     - **Procedural Memory:** Skills and procedures.

3. **Application in Travel Promotion Campaigns:**
   - The AI assistant suggested strategies to leverage these memory types to enhance the travel agency's services:
     - **Personalized Recommendations:** Use episodic memory for tailored travel suggestions.
     - **Enhanced Customer Support:** Implement RAG for accurate responses.
     - **Content Generation:** Use semantic memory for informative content.
     - **Automated Processes:** Utilize procedural memory for routine tasks.
     - **Feedback Analysis:** Use AI to identify and address common issues.

4. **Next Steps:**
   - The user decided to summarize the discussion and consult with the manager to seek approval for implementing these strategies.

### Actions:

1. **Conduct Sentiment Analysis:**
   - Regularly analyze customer feedback to gauge sentiment and identify areas for improvement.

2. **Explore AI Memory Systems:**
   - Investigate the use of RAG and different memory types to enhance AI capabilities in customer interactions.

3. **Develop Personalized Campaigns:**
   - Implement strategies for personalized travel recommendations and content generation.

4. **Enhance Customer Support:**
   - Use AI to provide more accurate and contextually relevant responses to customer inquiries.

5. **Automate Routine Tasks:**
   - Utilize AI to automate booking confirmations, itinerary updates, and follow-up communications.

6. **Analyze Feedback:**
   - Continuously analyze customer feedback to refine services and address concerns proactively.

7. **Consult Management:**
   - Present the proposed strategies to the manager for approval and potential implementation.